In [ ]:
import random
import faiss
import numpy as np
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer

In [ ]:
CONFIG = {
    "prefix_type":          "search",
    "hard_neg_rank_start":  10,
    "hard_neg_rank_end":    50,
    "hard_neg_count":       5,
    "max_triplets":         100_000,
    "log_every":            500,
    "max_tokens":           1024,
    "margin_tau":           0.01,
}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("deepvk/USER2-base")
model = SentenceTransformer("deepvk/USER2-base")

def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text, truncation=False))

def prompt_name(is_query: bool) -> str:
    return "search_query" if CONFIG["prefix_type"] == "search" and is_query else "search_document" if CONFIG["prefix_type"] == "search" else "clustering"
    

In [ ]:
def create_triplets(split, q_field: str, p_field: str, max_tokens: int = 500, margin_tau: float = CONFIG['margin_tau']):
    examples = [ex for ex in split if ex.get(q_field) and ex.get(p_field)]
    random.shuffle(examples)
    if not examples:
        return []

    unique_docs, doc2idx, ex2doc_idx = [], {}, {}
    for ex_id, ex in enumerate(tqdm(examples)):
        doc = ex[p_field]
        if count_tokens(doc) > max_tokens:
            continue
        if doc not in doc2idx:
            doc2idx[doc] = len(unique_docs)
            unique_docs.append(doc)
        ex2doc_idx[ex_id] = doc2idx[doc]

    if not unique_docs:
        return []

    doc_embs = model.encode(unique_docs, prompt_name=prompt_name(False), convert_to_numpy=True, show_progress_bar=True).astype("float32")
    faiss.normalize_L2(doc_embs)
    index = faiss.IndexFlatIP(doc_embs.shape[1])
    index.add(doc_embs)
    triplets = []

    for ex_id, ex in enumerate(tqdm(examples)):
        if len(triplets) >= CONFIG["max_triplets"]:
            break
        if ex_id and ex_id % CONFIG["log_every"] == 0:
            print(f"{ex_id}/{len(examples)} → {len(triplets)} triplets")

        q_text = ex[q_field]
        if count_tokens(q_text) > max_tokens:
            continue

        pos_idx = ex2doc_idx.get(ex_id)
        if pos_idx is None:
            continue
        pos_text = unique_docs[pos_idx]

        q_emb = model.encode(q_text, prompt_name=prompt_name(True), convert_to_numpy=True, show_progress_bar=False).astype("float32")
        q_emb_2d = q_emb.reshape(1, -1)
        faiss.normalize_L2(q_emb_2d)
        q_emb = q_emb_2d[0]

        pos_sim = float(np.dot(q_emb, doc_embs[pos_idx]))
        _, neigh = index.search(q_emb_2d, CONFIG["hard_neg_rank_end"] + 1)

        candidates = [i for i in neigh[0][CONFIG["hard_neg_rank_start"]:CONFIG["hard_neg_rank_end"] + 1]
                      if i != pos_idx and float(np.dot(q_emb, doc_embs[i])) < pos_sim - margin_tau]

        random.shuffle(candidates)
        neg_idxs = candidates[:CONFIG["hard_neg_count"]]

        if len(neg_idxs) < CONFIG["hard_neg_count"]:
            continue

        triplets.append({
            "query": q_text,
            "positive": pos_text,
            "negatives": [unique_docs[i] for i in neg_idxs]
        })

    unique_triplets = { (t["query"], t["positive"], tuple(t["negatives"])): t for t in triplets }
    return list(unique_triplets.values())

In [ ]:
def evaluate_hard_negative_quality(triplets, sample_size=CONFIG['sample_size']):
    sample = random.sample(triplets, k=min(sample_size, len(triplets)))

    q_texts   = [t["query"] for t in sample]
    pos_texts = [t["positive"] for t in sample]
    neg_texts = [n for t in sample for n in t["negatives"]]

    q_embs = model.encode(q_texts, prompt_name=prompt_name(True), convert_to_numpy=True, show_progress_bar=True)
    pos_embs = model.encode(pos_texts, prompt_name=prompt_name(False), convert_to_numpy=True, show_progress_bar=True)
    neg_embs = model.encode(neg_texts, prompt_name=prompt_name(False), convert_to_numpy=True, show_progress_bar=True)

    q_embs /= np.linalg.norm(q_embs, axis=1, keepdims=True)
    pos_embs /= np.linalg.norm(pos_embs, axis=1, keepdims=True)
    neg_embs /= np.linalg.norm(neg_embs, axis=1, keepdims=True)

    pos_sims = (q_embs * pos_embs).sum(axis=1)

    neg_sims, margins = [], []
    idx, bad_cnt = 0, 0
    for i, t in tqdm(enumerate(sample), total=len(sample)):
        n = len(t["negatives"])
        sims = q_embs[i] @ neg_embs[idx:idx + n].T

        hardest = sims.max()
        neg_sims.extend(sims.tolist())

        if hardest >= pos_sims[i]:
            bad_cnt += 1
        margins.append(pos_sims[i] - hardest)

        idx += n

    return {
        "sampled_triplets": len(sample),
        "mean_pos_sim": float(np.mean(pos_sims)),
        "mean_neg_sim": float(np.mean(neg_sims)),
        "mean_margin_pos_vs_hardest": float(np.mean(margins)),
        "triplets_with_harder_negative": bad_cnt
    }

def main():
    triplets = [...] 

    if not triplets:
        raise RuntimeError("Triplets list empty!")

    stats = evaluate_hard_negative_quality(triplets)

    print(f"""
    Sampled {stats['sampled_triplets']} triplets
    Mean cos-sim(query, positive): {stats['mean_pos_sim']:.4f}
    Mean cos-sim(query, negatives): {stats['mean_neg_sim']:.4f}
    Mean margin (pos - hardest neg): {stats['mean_margin_pos_vs_hardest']:.4f}
    Triplets with harder negative: {stats['triplets_with_harder_negative']}
    """)

if __name__ == "__main__":
    main()